In [100]:
import pandas as pd

doubleSpaces = [4,6,8]
doubleIndices = [11,12,13]

class Board():
    # creates a standard board
    # first position is not usable
    board = None
    prevNumMoved = None
    totalMoves = 0

    def __init__(self):
        self.board = [0 for i in range(14)]
        for i in range(2,10): self.board[i] = i
        self.board[0] = -1
        self.board[10] = 1

    def display(self):
        a = pd.Series(self.board[1:11])
        b = pd.Series([' ',' ',' ', self.board[11], ' ', self.board[12], ' ', self.board[13], ' ', ' '])
        df = pd.DataFrame([b,a])
        return df.to_string(index=False, header=False)
        
    def checkDown(self, i):
        x = self.board.index(i)
        if self.board.index(i) in doubleIndices:
            if self.board[ doubleSpaces[ doubleIndices.index(x) ] ] == 0: 
                return (True, doubleSpaces[ doubleIndices.index(x) ] )

        return (False, None)

    def checkUp(self, i):
        x = self.board.index(i)
        if self.board.index(i) in doubleSpaces:
            if self.board[ doubleIndices[ doubleSpaces.index(x) ] ] == 0: 
                return (True, doubleIndices[ doubleSpaces.index(x) ] )

        return (False, None)

    def checkLeft(self, i):
        if self.board[i-1] == 0: return (True, i-1)

        return (False, None)

    def checkRight(self, i):
        if self.board[i+1] == 0: return (True, i+1)

        return (False, None)

    def possibleMoves(self):
        moves = []
        for i in range(1,10):
            x = self.board.index(i)
            if self.checkUp(i)[0]: moves.append([i, self.checkUp(i)[1]])
            if self.checkDown(i)[0]: moves.append([i, self.checkDown(i)[1]])
            if self.checkLeft(i)[0]: moves.append([i, self.checkLeft(i)[1]])
            if self.checkRight(i)[0]: moves.append([i, self.checkRight(i)[1]])
        return moves

    # executes a valid move
    # move - tuple (number, new position)
    # board - list of size 13 expected
    def move(self, move):
        self.board[self.board.index(move[0])] = 0
        self.board[move[1]] = move[0]
        if self.prevNumMoved == move[0]:
            return
        else:
            self.prevNumMoved = move[0]
            self.totalMoves += 1



In [101]:
b = Board()
print(b.board, '\n\n')
b.move((4,11))
b.move((9,13))
b.move((8,9))
print(b.display())
b.checkDown(9)
b.possibleMoves()


[-1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 1, 0, 0, 0] 


      4   0   9    
0 2 3 0 5 6 7 0 8 1


TypeError: 'NoneType' object is not subscriptable